In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('basic').getOrCreate()

24/11/23 00:28:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/23 00:28:16 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
from pyspark.sql.functions import col,current_timestamp,count,countDistinct,sum,aggregate,desc,rank
from pyspark.sql.window import Window


In [3]:
races_df = spark.read.option('header',True).option('inferSchema',True).format('csv').load('../tbls/race/race.csv')\
    .filter('race_year = 2020 and name = "Abu Dhabi Grand Prix"')\
    .withColumnRenamed('name', 'race_name')\
    .withColumnRenamed('race_timestamp', 'race_date')\
    .select('circuiit_id','race_id',col('race_year'),col('race_name'),col('race_date'))


In [4]:
circuits_df = spark.read.option('header',True).option('inferSchema',True).format('csv').load('../tbls/circuits/circuits.csv')\
        .select('circuit_id',col('location').alias('circuit_location'))


In [5]:
drivers_df = spark.read.option('header',True).option('inferSchema',True).format('csv').load('../tbls/drivers/drivers.csv')\
    .withColumnRenamed('name','driver_name')\
    .withColumnRenamed('number','driver_number')\
    .withColumnRenamed('nationality','driver_nationality')\
    .select('driver_id','driver_name','driver_number','driver_nationality')


In [6]:
constructors_df = spark.read.option('header',True).option('inferSchema',True).format('csv').load('../tbls/constructors/constructors.csv')\
    .select('constructor_id',col('name').alias('team'))


In [7]:
results_df = spark.read.option('header',True).option('inferSchema',True).format('csv').load('../tbls/result/result.csv')\
    .select('race_id','result_id','driver_id','constructor_id','grid','fastest_lap',col('time').alias('race_time'),'points')


In [16]:
race_join = circuits_df.join(races_df, circuits_df.circuit_id == races_df.circuiit_id, 'inner')\
    .join(results_df,races_df.race_id == results_df.race_id,'inner')\
    .join(drivers_df,drivers_df.driver_id == results_df.driver_id, 'inner')\
    .join(constructors_df,constructors_df.constructor_id == results_df.constructor_id,'inner')

In [17]:
df1 = race_join .select('race_year','race_name','race_date','circuit_location','driver_name','driver_number','driver_nationality','team','grid','fastest_lap','race_time','points')\
    .withColumn('current_date',current_timestamp())\
    .orderBy(race_join.points.desc())


In [21]:
df1.coalesce(1).write.option('header',True).format('csv').save('../tbls/presentation/race_results')
